In [ ]:
import osmnx as ox
G = ox.graph_from_place("Manhattan, New York, USA", network_type="drive")
ox.save_graphml(G, "manhattan_drive.graphml")


In [ ]:
import networkx as nx
from geopy.geocoders import Nominatim
from opencage.geocoder import OpenCageGeocode
# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="myGeocoder")
api_key = 'b691284543b641248480ea652bbff899'
geocoder = OpenCageGeocode(api_key)

def get_coordinates(place_name):
    location = geolocator.geocode(place_name)
    if location:
        return (location.latitude, location.longitude)
    else:
        raise ValueError(f"Could not geocode location: {place_name}")

# Get the graph for the area (e.g., Manhattan)
G = ox.graph_from_place("Manhattan, New York, USA", network_type="drive")

# Define your locations by name
start_location = "Empire State Building, New York, NY"
end_location = "Times Square, New York, NY"

# Get coordinates for each location
start_coords = get_coordinates(start_location)
end_coords = get_coordinates(end_location)

# Find the nearest nodes in the graph to these coordinates
orig = ox.distance.nearest_nodes(G, X=start_coords[1], Y=start_coords[0])
dest = ox.distance.nearest_nodes(G, X=end_coords[1], Y=end_coords[0])

# Calculate the shortest path between nodes using A*
shortest_path = nx.shortest_path(G, orig, dest, weight="length")  # 'length' is the edge weight by default

# Plot the route
ox.plot_graph_route(G, shortest_path, route_linewidth=2, node_size=0)

GeocoderInsufficientPrivileges: Non-successful status code 403